In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [39]:
import seaborn as sns
import matplotlib.pyplot as plt 

from sklearn.ensemble import RandomForestClassifier


In [7]:
train = pd.read_csv('train.csv')
train.sample()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
34663,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Preprocession

In [8]:
y=train['label']
x=train.drop('label', axis=1)

## Grid Search

In [9]:
forest = RandomForestClassifier(random_state=42) # 깡통 모델 생성 

In [10]:
param_grid = [
    {'n_estimators' : [5], #[10,100,200] 넣어줄 수량의 콤비네이션을 딕셔너리 형식으로 만들어줌 
    'max_features' : [0.3, 0.4],  # 작으면 작을수록 랜덤한 성격이 들어간다, 전체 피쳐 중 몇 퍼센트만 셀렉해서 노드를 갈라줄 지 결정, none: 모든 피쳐 다 쓴다 
    'bootstrap' : [True,False]} # bagging 기본(복원추출), 비복원추출을 할 때는 false 옵션, 둘다 넣으면 두 경우 다 실험해봄(true, false)
] # 4*2 = 8 combination 형성 

In [11]:
grid_search = GridSearchCV(forest, #써치하고 싶은 하이퍼파라미터를 정해주고 콤비네이션을 써치하도록 해줌 (gridsearchcv)
                          param_grid,
                          cv=2, # 컴퓨터가 허락하는 대로 많이 할수록 좋지만, 연산량이 많이 늘어나니까 주의하기 
                          scoring='accuracy', # socring 옵션
                          return_train_score=True,
                           # train data에 대해서도 score를 쓸꺼냐?는 문제라서 크게 중요하지는 않다
                           #-> train data는 어차피 score가 높으니깐
                           #-> but, train과 test의 score가 너무 많이 다를 경우에는 중요할 수 있다
                           #(train만 점수가 높으면 train에만 맞춰서 fitting이 된 경우니깐)
                          n_jobs=-1, # -1은 내가 쓰고있는 코어를 다 쓰겠다(내가 쓸 수 있는 최대의 수) - 동시에 돌아가는 작업 수 
                          verbose=1) # 트레이닝 잘 되는 지 보고 싶을 때 verbose 

In [12]:
grid_search.fit(x,y)

Fitting 2 folds for each of 4 candidates, totalling 8 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:   23.8s finished


GridSearchCV(cv=2, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False, random_state=42,
                                              verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'b

In [13]:
grid_search.best_params_ #가장 scoring이 높게 나온 파라미터 

{'bootstrap': False, 'max_features': 0.3, 'n_estimators': 5}

In [23]:
grid_search.best_estimator_

RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
                       max_depth=None, max_features=0.3, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=5,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [27]:
grid_search.best_estimator_.predict(x) # 새롭게 트레이닝 할 필요없이 바로 predict로 적용할 수 있음 

array([1, 0, 1, ..., 7, 6, 9])

In [14]:
grid_search.cv_results_ 

{'mean_fit_time': array([ 7.7843411 ,  9.7360096 ,  9.51409411, 11.06682396]),
 'std_fit_time': array([0.03730679, 0.02723742, 0.17459202, 0.01291013]),
 'mean_score_time': array([0.12049341, 0.11996496, 0.11369455, 0.0814817 ]),
 'std_score_time': array([0.01582646, 0.01485693, 0.02230251, 0.01169062]),
 'param_bootstrap': masked_array(data=[True, True, False, False],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_max_features': masked_array(data=[0.3, 0.4, 0.3, 0.4],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[5, 5, 5, 5],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'bootstrap': True, 'max_features': 0.3, 'n_estimators': 5},
  {'bootstrap': True, 'max_features': 0.4, 'n_estimators': 5},
  {'bootstrap': False, 'max_features': 0.3, 'n_estimators': 5},
  {'boo

In [15]:
cvres = grid_search.cv_results_

for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score,params)

0.9085 {'bootstrap': True, 'max_features': 0.3, 'n_estimators': 5}
0.9074047619047619 {'bootstrap': True, 'max_features': 0.4, 'n_estimators': 5}
0.9190238095238096 {'bootstrap': False, 'max_features': 0.3, 'n_estimators': 5}
0.9165714285714286 {'bootstrap': False, 'max_features': 0.4, 'n_estimators': 5}


## Random Search

In [16]:
from scipy.stats import uniform as sp_uniform # float uniform dist
from scipy.stats import randint as sp_randint # int uniform dist

In [17]:
param_dist = {'max_features' : sp_uniform(0.3, 0.5), #uniform dist (동일한 확률로 숫자를 뽑아라)
             'bootstrap' : [True, False], 
             'criterion' : ['gini','entropy'],
             'n_estimators' : [5],
             'max_depth' : sp_randint(5,25) } #5와 25사이에서 랜덤 숫자를 뽑아라 
# grid search는 값을 주고 뽑았다면, random search는 랜덤하게 주고 뽑음 

In [18]:
random_search = RandomizedSearchCV(forest, param_dist, n_iter=5, cv=2, verbose=1, n_jobs=-1)
# n_iter: 사실 많이 해야함 (공간이 넓어질수록 많이 해야함)
# n_jobs: 몇개의 코어를 썼냐 

In [19]:
random_search.fit(x,y)

Fitting 2 folds for each of 5 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   50.5s finished


RandomizedSearchCV(cv=2, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs=None

In [20]:
for i in random_search.best_estimator_.estimators_: #estimators를 검색하면 가지고 있는 leaf node 중에서 depth를 알아보고 싶어서.
    print(i.get_depth())

23
23
23
23
23


In [26]:
random_search.best_estimator_.estimators_[0].predict(x)

array([1., 0., 1., ..., 7., 6., 9.])

# 선미님이 찾은 최적의 파라미터
- 코드 복사 
- 0.9571904761904761 {'bootstrap': True, 'criterion': 'entropy', 'max_depth': 19, 'max_features': 0.6659969709057025, 'n_estimators': 100}


In [40]:
forest = RandomForestClassifier(random_state=42,
                                 max_features = 0.6659969709057025,  
                                 bootstrap = True , 
                                 criterion = 'entropy',
                                 n_estimators = 100,
                                 max_depth = 19,
                                 verbose=1
                               )

In [41]:
forest.fit(x,y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  6.2min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=19, max_features=0.6659969709057025,
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=100, n_jobs=None, oob_score=False,
                       random_state=42, verbose=1, warm_start=False)

In [61]:
prediction = forest.predict(x)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.0s finished


In [45]:
prediction_proba = forest.predict_proba(x)[:,1]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.9s finished


In [47]:
y = pd.DataFrame(y)
y.columns = ['Actual']

In [48]:
y['Predict'] = prediction
y['Predict_proba'] = prediction_proba

In [49]:
y.head()

,Actual,Predict,Predict_proba
0,1,1,0.999703
1,0,0,0.000000
2,1,1,0.999923
3,4,4,0.000000
4,0,0,0.000000


In [62]:
prediction

array([1, 0, 1, ..., 7, 6, 9])

In [64]:
prediction = submission['Label']

In [66]:
submission

,ImageId,Label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
5,6,0
6,7,0
7,8,0
8,9,0
9,10,0


In [59]:
submission = pd.read_csv('sample_submission.csv')

In [67]:
submission.to_csv('submission_baseline.csv', index=False)